## Data Preparation für die Pandas-Analyseaufgabe
Zur Datenvorverarbeitung wird eine Kombination aus Pandas und SQL (mit SQLite) verwendet ('whatever works ...')

In [1]:
import pandas as pd

Quelle: [https://www.timeanddate.com/worldclock/distances.html?n=64]

**Anpassungen**
1. Spaltennamen
2. Einheiten bei den Entfernungen

In [2]:
chicago_dist = pd.read_clipboard()
chicago_dist.head()

,Location,Local time,Unnamed: 2,Distance,Unnamed: 4,Direction
0,USA - Illinois - Chicago *,Fri 09:12,-,-,-,NaN
1,USA - Illinois - Evanston *,Fri 09:12,19 km,12 miles,10 nm,North-northwest NNW
2,USA - Indiana - Hammond *,Fri 09:12,32 km,20 miles,17 nm,South-southeast SSE
3,USA - Illinois - Orland Park *,Fri 09:12,34 km,21 miles,18 nm,South-southwest SSW
4,USA - Indiana - Gary *,Fri 09:12,40 km,25 miles,21 nm,Southeast SE


In [3]:
chicago_dist.to_csv('chicago_dist.csv', sep='|', encoding='utf-8') 
# dann im Editor nachbearbeiten ...

Mit *Suchen & Ersetzen* kann man im Editor am einfachsten die `Location` Spalte weiter aufteilen in `Country, State, City` und ferner noch die Spaltenüberschriften für die Distanzen anpassen in `Distance_km, Distance_mil, Distance_nm`. Außerdem kann man leicht die Entfernungseinheiten los werden und auch etwaige Leerzeichen ...

#### Nächster Schritt: US-Bundesstaaten mit Abkürzungen
Quelle: [http://bigdave44.com/features/the-mine/us-states-abbreviations-capitals-nicknames/]

In [5]:
us_states = pd.read_clipboard()
us_states.head()

,State,Abbrv.,Capital,Nickname
0,Alabama,AL,Montgomery,"Yellowhammer State, Heart of Dixie, Camellia S..."
1,Alaska,AK,Juneau,The Last Frontier
2,Arizona,AZ,Phoenix,"Grand Canyon State, Copper State"
3,Arkansas,AR,Little Rock,"The Natural State, Land of Opportunity, The Ra..."
4,California,CA,Sacramento,Golden State


In [6]:
us_states.to_csv('us_states.csv', sep='|', encoding='utf-8')

#### Weitere Schritte
Die (bearbeiteten) `csv`-Dateien werden dann in eine SQLite-Datenbank `chicago_dist.db`importiert. Aus der Tabelle `chicago_dist` werden anschließend alle Informationen entfernt, die sich nicht auf Kanada oder die USA beziehen. Ferner wird geprüft, ob es zu allen US-Bundesstaaten mindestens eine Entfernungsangabe gibt. Es stellt sich dann heraus, dass dies für Oregon nicht der Fall ist.

Fehlende Entfernung nach Oregon: [http://www.distance-cities.com/distance-portland-or-to-chicago-il]

Diese fehlende Entfernung wird mit einem `INSERT`-Statement eingefügt und schließlich die folgende Sicht erzeugt, die man später im Rahmen der Übungsaufgabe einlesen kann:
```SQL
CREATE VIEW state_to_chicago AS
SELECT Abbrv AS state, ROUND(AVG(Distance_mil),0) AS distance
FROM us_states us LEFT JOIN chicago_dist cd ON us.State = cd.State
GROUP BY Abbrv;
```